In [49]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms, models
import torch.nn.functional as F

import helper

We need to arrange the images in such a format that each class should have it's own directory.

In [43]:
data_dir = 'Cat_Dog_data'

data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [66]:
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [67]:
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [68]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [69]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5
train_losses, test_losses =[],[]
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))     
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

Epoch 1/1.. Train loss: 0.728.. Test loss: 0.684.. Test accuracy: 0.564
Epoch 1/1.. Train loss: 0.729.. Test loss: 0.686.. Test accuracy: 0.556
Epoch 1/1.. Train loss: 0.722.. Test loss: 0.688.. Test accuracy: 0.545
Epoch 1/1.. Train loss: 0.713.. Test loss: 0.688.. Test accuracy: 0.543
Epoch 1/1.. Train loss: 0.718.. Test loss: 0.689.. Test accuracy: 0.531
Epoch 1/1.. Train loss: 0.728.. Test loss: 0.688.. Test accuracy: 0.534
Epoch 1/1.. Train loss: 0.728.. Test loss: 0.694.. Test accuracy: 0.532
Epoch 1/1.. Train loss: 0.722.. Test loss: 0.688.. Test accuracy: 0.534
Epoch 1/1.. Train loss: 0.724.. Test loss: 0.694.. Test accuracy: 0.517
Epoch 1/1.. Train loss: 0.720.. Test loss: 0.692.. Test accuracy: 0.525
Epoch 1/1.. Train loss: 0.724.. Test loss: 0.692.. Test accuracy: 0.517
Epoch 1/1.. Train loss: 0.727.. Test loss: 0.694.. Test accuracy: 0.519
Epoch 1/1.. Train loss: 0.732.. Test loss: 0.689.. Test accuracy: 0.530
Epoch 1/1.. Train loss: 0.729.. Test loss: 0.692.. Test accuracy

Epoch 1/1.. Train loss: 0.685.. Test loss: 0.691.. Test accuracy: 0.527
Epoch 1/1.. Train loss: 0.679.. Test loss: 0.689.. Test accuracy: 0.539
Epoch 1/1.. Train loss: 0.666.. Test loss: 0.692.. Test accuracy: 0.521
Epoch 1/1.. Train loss: 0.677.. Test loss: 0.690.. Test accuracy: 0.540
Epoch 1/1.. Train loss: 0.672.. Test loss: 0.694.. Test accuracy: 0.533
Epoch 1/1.. Train loss: 0.675.. Test loss: 0.693.. Test accuracy: 0.540
Epoch 1/1.. Train loss: 0.684.. Test loss: 0.691.. Test accuracy: 0.534
Epoch 1/1.. Train loss: 0.681.. Test loss: 0.691.. Test accuracy: 0.535
Epoch 1/1.. Train loss: 0.673.. Test loss: 0.692.. Test accuracy: 0.528
Epoch 1/1.. Train loss: 0.679.. Test loss: 0.694.. Test accuracy: 0.525
Epoch 1/1.. Train loss: 0.685.. Test loss: 0.694.. Test accuracy: 0.529
Epoch 1/1.. Train loss: 0.675.. Test loss: 0.691.. Test accuracy: 0.527
Epoch 1/1.. Train loss: 0.673.. Test loss: 0.694.. Test accuracy: 0.521
Epoch 1/1.. Train loss: 0.685.. Test loss: 0.693.. Test accuracy

In [70]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

In [71]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)

NameError: name 'train_losses' is not defined